In [ ]:
# This project uses GluonTS, a Python toolkit for probabilistic time series modeling.
# Copyright 2019 Amazon.com, Inc. or its affiliates. All Rights Reserved.
# Licensed under the Apache License, Version 2.0 (the "License").
# You may not use this file except in compliance with the License.
# A copy of the License is located at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
# or in the "license" file accompanying this file. This file is distributed 
# on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, 
# either express or implied. See the License for the specific language 
# governing permissions and limitations under the License.

import torch
from gluonts.dataset.common import ListDataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.torch.model.predictor import PyTorchPredictor
from torch.utils.data import DataLoader
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import time

# Set PyTorch Matmul precision
torch.set_float32_matmul_precision('medium')

# File path settings
train_pkl_path = 'F:/DeepAR_models/Data_clean/gluonts_listDataset_deepar_validation_dataframe.pkl'
model_load_path = 'F:/DeepAR_models/Model'  
output_csv_path = 'F:/DeepAR_models/Data_Plotly/forecast_results_all.csv'

# Create CSV file
def save_forecasts_to_csv(forecasts, tss, output_path):
    forecast_data = []
    for i, (forecast, series) in enumerate(zip(forecasts, tss)):
        for j in range(len(forecast.samples[0])):
            forecast_data.append({
                "item_id": i,
                "date": (series.index[-1] + j + 1).to_timestamp(),
                "forecast_mean": forecast.mean[j],
                "forecast_quantile_0.1": forecast.quantile(0.1)[j],
                "forecast_quantile_0.9": forecast.quantile(0.9)[j]
            })

    forecast_df = pd.DataFrame(forecast_data)
    forecast_df.to_csv(output_path, index=False)
    print(f"Forecast results have been saved to {output_path}")

# Determine device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device_index = torch.cuda.current_device() if torch.cuda.is_available() else None
print(f"Using device: {device}")

# Check if CUDA is available
if torch.cuda.is_available():
    print(f"CUDA is available, using device: {torch.cuda.get_device_name(device_index)}")
else:
    print("CUDA is not available, using CPU")

# Set GPU memory usage fraction to 90%
if device.type == 'cuda':
    torch.cuda.set_per_process_memory_fraction(0.9, device_index)

# Data loading
def load_pkl(pkl_path):
    with open(pkl_path, 'rb') as f:
        return pd.read_pickle(f)

print("Loading data...")
train_data = load_pkl(train_pkl_path)

# Set up PyTorch DataLoader, enable multi-threaded data loading
def from_gluonts(dataset):
    for entry in dataset:
        target = torch.tensor(entry['target'].tolist(), dtype=torch.float).to(device)
        feat_dynamic_real = torch.tensor(entry['feat_dynamic_real'], dtype=torch.float).to(device)
        yield target, feat_dynamic_real  # Return target and feat_dynamic_real to ensure data is moved to the GPU

train_loader = DataLoader(from_gluonts(train_data), batch_size=512, num_workers=8)

# Check current free and total GPU memory
if device.type == 'cuda':
    free_memory, total_memory = torch.cuda.mem_get_info(device_index)
    print(f"Free memory: {free_memory / 1024**2:.2f} MB, Total memory: {total_memory / 1024**2:.2f} MB")

# Load pre-trained model
print("Loading model from disk...")
try:
    predictor = PyTorchPredictor.deserialize(Path(model_load_path))
    print("Model loaded and moved to GPU.")

    print("Starting prediction...")
    forecast_it, ts_it = make_evaluation_predictions(dataset=train_data, predictor=predictor, num_samples=100)
    print("Prediction completed.")

    forecasts = list(forecast_it)
    tss = list(ts_it)
    print(f"Generated {len(forecasts)} forecast results.")

    for i, (forecast, series) in enumerate(zip(forecasts, tss)):
        print(f"Forecast result {i+1}:")
        print(f"Actual data:\n{series}")
        print(f"Forecast mean: {forecast.mean}")
        print(f"Number of forecast samples: {len(forecast.samples)}")
        print(f"90% confidence interval: {forecast.quantile(0.1)}, {forecast.quantile(0.9)}")
        print("\n")

    # Independent CSV generation process
    save_forecasts_to_csv(forecasts, tss, output_csv_path)
    
    # Start evaluating forecast results
    print("Starting evaluation of forecast results...")
    evaluator = Evaluator()
    try:
        for _ in tqdm(range(1), desc="Evaluating forecast results"):
            agg_metrics, item_metrics = evaluator(tss, forecasts)
        print("Evaluation completed.")

        print("Aggregate metrics:")
        for key, value in agg_metrics.items():
            print(f"{key}: {value}")
    except Exception as e:
        print(f"An exception occurred during evaluation: {e}")
except Exception as e:
    print(f"An exception occurred during prediction or evaluation: {e}")
